In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import sqlite3


In [ ]:
data = pd.read_html("foggia_2024.html", encoding="utf-8", decimal=".")
data = data[0]

In [ ]:
def load_db_sql(table_name: str = 'stats', db_name="fm_estatistica.db") -> pd.DataFrame:
    
    conn = sqlite3.connect('fm_estatistica.db')
    
    
    try:
        df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
        # debug_print(Fore.GREEN + f"[INFO]: df carregada com sucesso." + Style.RESET_ALL)
        
        try:
            df['posicao_analise'] = df['posicao_analise'].apply(lambda x: x.split("."))
            # debug_print(Fore.GREEN + f"[INFO]: Coluna 'posicao_analise' convertida com sucesso." + Style.RESET_ALL)
            
        except Exception as e:
            pass
            # debug_print(Fore.RED + f"[ERROR]: Erro ao converter coluna 'posicao_analise': {str(e)}" + Style.RESET_ALL)
        
        return df
    
    except Exception as e:
        # debug_print(Fore.RED + f"[ERROR]: Erro ao carregar df: {str(e)}" + Style.RESET_ALL)
        
        return pd.DataFrame()

In [ ]:
df_proc = load_db_sql()

In [ ]:
df_proc.to_excel(excel_writer="data.xlsx",index=False)

In [ ]:
df_goleadores = df_proc[df_proc['gols'] >= 15]

In [ ]:
colunas = [
    'nome',
    'clube_atual',
    'gols_esperados_sem_penaltis_p90',
    'gols_sem_penaltis_p90',
    'gols_esperados_sem_penalti_por_chutes_sem_penalti',
    'chutes_sem_penaltis_p90',
    'chutes_no_gol_sem_penalti_p90',
    'perigo_das_finalizacoes'
    ]

colunas_rename = {
    'gols_sem_penaltis_p90':'npg',
    'gols_esperados_sem_penaltis_p90':'xnpg',
    'gols_esperados_sem_penalti_por_chutes_sem_penalti':'xnpg_per_shot',
    'chutes_sem_penaltis_p90':'np_sht_p90',
    'chutes_no_gol_sem_penalti_p90':'np_shtG_p90',
    }

In [ ]:
df_goleadores[colunas].sort_values(by='gols_sem_penaltis_p90', ascending=False).rename(columns=colunas_rename)

In [ ]:
import re
def _get_season(path:str) -> int:
    
    pattern = r'^(?P<nome>.+_*)(?P<season>[0-9]{4})(?P<ext>.{1}[a-zA-Z0-9]+)'
    season = re.search(pattern, path).group('season')
    
    return int(season)

In [ ]:

path = 'Italia_254024.html'

_get_season(path)

In [ ]:
import pandas as pd

data = pd.read_html('italia_2024.html', encoding="utf-8", decimal=".")
df = pd.DataFrame(data[0])

In [ ]:
df

In [2]:
from data_manipulation import*
from data_preprocessing import*
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

_df = fm_create_dataframe(path='teste_2024.html')

True

In [ ]:
_df['np_chutes_gol_p90'].where(_df['divisao'].str.contains('Serie C NOW Girone',case=False,na=False)).describe()

In [ ]:
_df[['nome','idade','posicao_analise','divisao','aval_fin','npxG_p90','npG_p90','npxG_per_np_chute']].where(_df['divisao'].str.contains('Serie C NOW Girone',case=False,na=False)).sort_values(by=['aval_fin'],ascending=False)

In [ ]:
from fm_analycts_parametrs import DEF_STATS
ordem = ['nome', 'clube','divisao','posicao_analise','idade','minutos'] + DEF_STATS
_df = _df[ordem].sort_values(by=['aval_def'],ascending=False)

In [5]:
_df['divisao'].unique().tolist()

['Serie C NOW Girone C', 'Serie C NOW Girone B', 'Serie C NOW Girone A']

In [ ]:
div_med = _df.groupby('divisao')['aval_def'].mean()
div_desv = _df.groupby('divisao')['aval_def'].std()

In [1]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

NameError: name '_df' is not defined

In [12]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

_df_zagueiros['aval_def_adj'] = _df_zagueiros.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_adef_adj'] = _df_zagueiros.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_des_adj'] = _df_zagueiros.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_duel_adj'] = _df_zagueiros.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_jg_ar_adj'] = _df_zagueiros.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

_df_zagueiros['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df_zagueiros['aval_def_adj1'] = (((     (_df_zagueiros['rtg_jg_ar_adj']*_df_zagueiros['jg_ar_t_p90']*p1) +
                                        _df_zagueiros['rtg_duel_adj']*p2 + 
                                        (_df_zagueiros['rtg_des_adj']*_df_zagueiros['des_t_p90'])*p3 + 
                                        (_df_zagueiros['poss_g_p90']/_df_zagueiros['adef_t_p90'])*p4+
                                        _df_zagueiros['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df_zagueiros['aval_def_adj1'] = _df_zagueiros['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df_zagueiros['aval_def_adj2'] = (_df_zagueiros['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

print(_df_zagueiros.groupby('divisao')['rtg_adef'].mean())
print(_df_zagueiros.groupby('divisao')['rtg_adef'].std())

_df_zagueiros[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df_zagueiros[(_df_zagueiros['divisao'].str.contains('Serie C NOW')) & (_df_zagueiros['minutos'] >= 900) & (_df_zagueiros['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'minutos',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def'],ascending=False)

divisao
Serie C NOW Girone A    0.701029
Serie C NOW Girone B    0.693433
Serie C NOW Girone C    0.670641
Name: rtg_adef, dtype: float64
divisao
Serie C NOW Girone A    0.044054
Serie C NOW Girone B    0.047083
Serie C NOW Girone C    0.059856
Name: rtg_adef, dtype: float64


,nome,clube,divisao,minutos,posicao_analise,aval_def,aval_def_adj,aval_def_adj1,aval_def_adj2,rtg_adef,rtg_adef_adj,rtg_des,rtg_des_adj,rtg_duel,rtg_duel_adj,rtg_jg_ar,rtg_jg_ar_adj
180,Giovanni Errico,Altamura,Serie C NOW Girone C,1284,[Zagueiro],2.57,2.610792,4.51,1.1275,0.77,1.659976,1.14,1.486627,0.54,1.691954,1.03,1.823075
6,Omar Khailoti,Novara,Serie C NOW Girone A,1250,"[Lateral-Esquerdo, Zagueiro]",2.55,1.681118,2.54,0.6350,0.70,-0.023367,0.92,-0.548680,0.54,1.086099,0.97,1.421767
196,Edoardo Grosso,Sestri Levante,Serie C NOW Girone B,2237,[Zagueiro],2.50,1.753329,4.28,1.0700,0.77,1.626211,1.03,0.201882,0.49,0.350328,1.00,1.961415
208,Filippo Pellacani,Pescara,Serie C NOW Girone B,1179,[Zagueiro],2.50,1.753329,3.71,0.9275,0.73,0.776650,0.89,-0.952785,0.47,-0.040872,0.99,1.826769
119,Mario Noce,Legnago,Serie C NOW Girone A,2639,"[Lateral-Direito, Zagueiro]",2.49,1.425758,3.81,0.9525,0.77,1.565608,0.94,-0.394186,0.51,0.525177,0.99,1.739794
64,Gabriele Berto,Atalanta,Serie C NOW Girone A,1558,"[Lateral-Direito, Zagueiro]",2.45,1.255519,2.72,0.6800,0.73,0.657622,1.00,0.069295,0.48,-0.035745,0.96,1.262754
186,Matteo Salvi,Clodiense,Serie C NOW Girone A,2696,[Zagueiro],2.45,1.255519,1.23,0.3075,0.74,0.884619,1.04,0.378282,0.53,0.899125,0.90,0.308673
112,Laurens Serpe,Pro Vercelli,Serie C NOW Girone A,3487,[Zagueiro],2.45,1.255519,2.37,0.5925,0.73,0.657622,0.94,-0.394186,0.48,-0.035745,0.95,1.103740
169,Sheriff Kassama,Trento,Serie C NOW Girone A,1979,"[Lateral-Direito, Zagueiro]",2.44,1.212959,2.39,0.5975,0.72,0.430626,1.04,0.378282,0.51,0.525177,0.95,1.103740
45,Luka Koblar,Novara,Serie C NOW Girone A,3309,[Zagueiro],2.44,1.212959,2.16,0.5400,0.75,1.111615,1.20,1.614232,0.53,0.899125,0.92,0.626700


In [ ]:
_df['aval_def_adj'] = _df.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_adef_adj'] = _df.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_des_adj'] = _df.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_duel_adj'] = _df.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_jg_ar_adj'] = _df.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
_df['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df['aval_def_adj1'] = (((     (_df['rtg_jg_ar_adj']*_df['jg_ar_t_p90']*p1) +
                                        _df['rtg_duel_adj']*p2 + 
                                        (_df['rtg_des_adj']*_df['des_t_p90'])*p3 + 
                                        (_df['poss_g_p90']/_df['adef_t_p90'])*p4+
                                        _df['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df['aval_def_adj1'] = _df['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df['aval_def_adj2'] = (_df['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

In [ ]:
_df[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df[(_df['divisao'] == 'Serie A TIM') & (_df['minutos'] >= 900) & (_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def_adj2'],ascending=False)

In [ ]:
_df[ordem].sort_values(by=['aval_def'],ascending=False)